# Wrangle Data

### Acquire

In [1]:
import numpy as np
import pandas as pd
import env

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
logs_query = """
SELECT date, time, path, user_id, name AS 'cohort', ip, start_date, end_date, program_id
FROM logs LEFT JOIN cohorts ON cohorts.id = logs.cohort_id;
"""

In [3]:
logs_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'

In [4]:
logs = pd.read_sql(logs_query, logs_url)

In [5]:
logs.head()

,date,time,path,user_id,cohort,ip,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,2.0


In [6]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort      847330 non-null  object 
 5   ip          900223 non-null  object 
 6   start_date  847330 non-null  object 
 7   end_date    847330 non-null  object 
 8   program_id  847330 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 61.8+ MB


### Prepare

In [7]:
#Create new dataframe for program_id
program = {
    'id':[1,2,3,4],
    'program':['PHP Full Stack Web Development',
               'Java Full Stack Web Development',
               'Data Science',
               'Front End Web Development'],
    'subdomain': ['php', 'java', 'ds', 'fe']
}

In [8]:
program = pd.DataFrame(program)
program.head()

,id,program,subdomain
0,1,PHP Full Stack Web Development,php
1,2,Java Full Stack Web Development,java
2,3,Data Science,ds
3,4,Front End Web Development,fe


In [9]:
#Join new dataframe with logs dataframe
logs = logs.merge(program, how = 'left', left_on = 'program_id', right_on = 'id').drop(columns = ['id', 'program_id'])

In [10]:
logs.head()

,date,time,path,user_id,cohort,ip,start_date,end_date,program,subdomain
0,2018-01-26,09:55:03,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php
1,2018-01-26,09:56:02,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php
4,2018-01-26,09:56:24,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java


In [11]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        900223 non-null  object
 1   time        900223 non-null  object
 2   path        900222 non-null  object
 3   user_id     900223 non-null  int64 
 4   cohort      847330 non-null  object
 5   ip          900223 non-null  object
 6   start_date  847330 non-null  object
 7   end_date    847330 non-null  object
 8   program     847330 non-null  object
 9   subdomain   847330 non-null  object
dtypes: int64(1), object(9)
memory usage: 75.5+ MB


In [12]:
#Convert date to datetime object and create new month, year, and day columns
logs.date = pd.to_datetime(logs.date)

In [13]:
logs['year'] = logs.date.dt.year

In [14]:
logs['month'] = logs.date.dt.month_name()

In [15]:
logs['day'] = logs.date.dt.day_name()

In [16]:
logs.head()

,date,time,path,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day
0,2018-01-26,09:55:03,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
1,2018-01-26,09:56:02,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
4,2018-01-26,09:56:24,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday


In [17]:
logs.head()

,date,time,path,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day
0,2018-01-26,09:55:03,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
1,2018-01-26,09:56:02,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday
4,2018-01-26,09:56:24,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday


In [18]:
#Convert the time column to a datetime object and create a new column for hour
logs.time = pd.to_datetime(logs.time)

In [19]:
logs['hour'] = logs.time.dt.hour

In [20]:
logs.head()

,date,time,path,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day,hour
0,2018-01-26,2021-10-19 09:55:03,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9
1,2018-01-26,2021-10-19 09:56:02,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9
2,2018-01-26,2021-10-19 09:56:05,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9
3,2018-01-26,2021-10-19 09:56:06,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9
4,2018-01-26,2021-10-19 09:56:24,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday,9


In [21]:
# I don't think the time column will be useful now, so go ahead and drop it
logs = logs.drop(columns = ['time'])

In [22]:
#Create a column to determine whether or not the user is a graduate
logs['is_graduate'] = (logs.date > logs.end_date) & (logs.cohort != 'Staff')

In [23]:
logs.head()

,date,path,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day,hour,is_graduate
0,2018-01-26,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True
1,2018-01-26,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True
2,2018-01-26,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True
3,2018-01-26,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True
4,2018-01-26,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday,9,False


In [24]:
logs['current_student'] = (logs.date <= logs.end_date) & (logs.cohort != 'Staff')

In [25]:
logs.head()

,date,path,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day,hour,is_graduate,current_student
0,2018-01-26,/,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True,False
1,2018-01-26,java-ii,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True,False
2,2018-01-26,java-ii/object-oriented-programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True,False
3,2018-01-26,slides/object_oriented_programming,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,9,True,False
4,2018-01-26,javascript-i/conditionals,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday,9,False,True


In [26]:
request_path_and_params = logs.path.str.split('/', expand=True)
request_path_and_params.columns = ['request_section', 'request_subject', 'request_lesson', 'param_4', 'param_5', 'param_6', 'param_7', 'param_8']
logs = logs.drop(columns='path').join(request_path_and_params)

In [27]:
logs.head()

,date,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,...,is_graduate,current_student,request_section,request_subject,request_lesson,param_4,param_5,param_6,param_7,param_8
0,2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,...,True,False,,,None,None,None,None,None,None
1,2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,...,True,False,java-ii,None,None,None,None,None,None,None
2,2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,...,True,False,java-ii,object-oriented-programming,None,None,None,None,None,None
3,2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,...,True,False,slides,object_oriented_programming,None,None,None,None,None,None
4,2018-01-26,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,...,False,True,javascript-i,conditionals,None,None,None,None,None,None


In [28]:
#Now convert 'date' to the index
logs = logs.set_index('date').sort_index()

In [29]:
logs.head()

,user_id,cohort,ip,start_date,end_date,program,subdomain,year,month,day,...,is_graduate,current_student,request_section,request_subject,request_lesson,param_4,param_5,param_6,param_7,param_8
date,,,,,,,,,,,,,,,,,,,,,
2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,...,True,False,,,None,None,None,None,None,None
2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,...,True,False,java-ii,None,None,None,None,None,None,None
2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,...,True,False,java-ii,object-oriented-programming,None,None,None,None,None,None
2018-01-26,1,Hampton,97.105.19.61,2015-09-22,2016-02-06,PHP Full Stack Web Development,php,2018,January,Friday,...,True,False,slides,object_oriented_programming,None,None,None,None,None,None
2018-01-26,2,Teddy,97.105.19.61,2018-01-08,2018-05-17,Java Full Stack Web Development,java,2018,January,Friday,...,False,True,javascript-i,conditionals,None,None,None,None,None,None


In [30]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          900223 non-null  int64 
 1   cohort           847330 non-null  object
 2   ip               900223 non-null  object
 3   start_date       847330 non-null  object
 4   end_date         847330 non-null  object
 5   program          847330 non-null  object
 6   subdomain        847330 non-null  object
 7   year             900223 non-null  int64 
 8   month            900223 non-null  object
 9   day              900223 non-null  object
 10  hour             900223 non-null  int64 
 11  is_graduate      900223 non-null  bool  
 12  current_student  900223 non-null  bool  
 13  request_section  900222 non-null  object
 14  request_subject  731934 non-null  object
 15  request_lesson   226870 non-null  object
 16  param_4          26309 non-null   object

In [32]:
logs.isnull().sum()

user_id                 0
cohort              52893
ip                      0
start_date          52893
end_date            52893
program             52893
subdomain           52893
year                    0
month                   0
day                     0
hour                    0
is_graduate             0
current_student         0
request_section         1
request_subject    168289
request_lesson     673353
param_4            873914
param_5            898295
param_6            899195
param_7            900209
param_8            900222
dtype: int64

There are 52,893 entries that are concerning. They seem to have no affiliation with any cohort and are not labeled as staff. I should take a closer look at these. 

As far as the nulls for the path parameters, those are acceptable. Although, the ones with the most null values would be worth investigating a little more closely.

I will not be imputing those null values since they could be anomalous.

In [37]:
logs.cohort.value_counts().nunique()

47